# Advanced Lane Finding Project
### The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
%matplotlib qt

### Step 1: Camera Calibration

In [7]:
img = cv2.imread('test_images/test1.jpg')
img_size = (img.shape[1], img.shape[0])

In [2]:
### Step 1 camera calibration ###

def calibration():
    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = plt.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    # Do camera calibration given object points and image points
    # img_size = (img.shape[1], img.shape[0])
    # ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    return mtx, dist

### Step 2: color transforms, gradients, etc. to create a thresholded binary image.

In [3]:
### Step 2: Pipeline for binary transform, Thresholding and birds eye prespective
#SobelX + perspective transform

image = mpimg.imread('test_images/straight_lines1.jpg')

# Edit this function to create your own pipeline.
def pipeline(img):
    #img = np.copy(img)
    
    h_thresh = (10, 25)
    s_thresh = (90, 255)
    sx_thresh = (30, 255)
    
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold color channel
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    white_binary = np.zeros_like(gray)
    white_binary[(gray > 200) & (gray <= 255)] = 1
    
    # Stack each channel
    binary_1 = cv2.bitwise_or(sxbinary, white_binary)
    binary_2 = cv2.bitwise_or(h_binary, s_binary)
    combined_binary = cv2.bitwise_or(binary_1, binary_2)
    
    #color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    #combined_binary = np.zeros_like(sxbinary)
    #combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    #warp_result, Minv = warp(combined_binary)
    return combined_binary
    
combined_binary = pipeline(image)

## check ##
#out_img = np.dstack((combined_binary, combined_binary, combined_binary))*255
#plt.imshow(out_img, cmap = 'gray')

# Plot the result
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()

# ax1.imshow(image)
# ax1.set_title('Original Image', fontsize=40)

# ax2.imshow(binary_warped, cmap = 'gray')
# ax2.set_title('Pipeline Result', fontsize=40)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Step 3: Prespective transform

In [14]:
### Step 3: Prespective transform
mtx, dist = calibration()
def warp(img):
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    offset = 300
    
    # Source points   1. bottom left, 2. top left, 3. top right, 4. bottom right
    src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
    
    # destination points
    dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])
    
    # Computing perspective transform M with src and dst points
    M = cv2.getPerspectiveTransform(src, dst)
    
    # Compute inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Computing wraped image using perspective transform, M, using linear interpolation
    warped = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)
    
    return warped, Minv

In [9]:
binary_warped, Minv = warp(combined_binary)

### Step 4 & 5: perspective transform to rectify binary image ("birds-eye view") and detect lane pixels and fit to find the lane boundary.

In [10]:
### Step 4 and 5: lane line detection using histogram and previous window details ###

def find_lane_pixels_histogram(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint


    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero (i.e. activated) pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []


    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        # (win_xleft_high,win_y_high),(0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),
        # (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # print("left lane inds", left_lane_inds)
    
    return leftx, lefty, rightx, righty, out_img


def fit_poly(binary_warped, leftx, lefty, rightx, righty):
    ### Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped[0]-1, binary_warped[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('Line fitting failed!!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

def finding_lane_pixels_previous_info(binary_warped):
    global previous_left_fit
    global previous_right_fit
    
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (previous_left_fit[0]*(nonzeroy**2) + previous_left_fit[1]*nonzeroy + 
                    previous_left_fit[2] - margin)) & (nonzerox < (previous_left_fit[0]*(nonzeroy**2) + 
                    previous_left_fit[1]*nonzeroy + previous_left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (previous_right_fit[0]*(nonzeroy**2) + previous_right_fit[1]*nonzeroy + 
                    previous_right_fit[2] - margin)) & (nonzerox < (previous_right_fit[0]*(nonzeroy**2) + 
                    previous_right_fit[1]*nonzeroy + previous_right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    return leftx, lefty, rightx, righty
    
    
def visualization(binary_warped, left_fitx, right_fitx, ploty):
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    margin = 100
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='green')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

# Run image through the pipeline
# Note that in your project, you'll also want to feed in the previous fits
#result = search_around_poly(binary_warped)

# View your output
#plt.imshow(result)

### Step 6: Determine the curvature of the lane and vehicle position with respect to center.

In [11]:
### Step 6 Calculating vehicle position and radius of curvature on road ###

def measure_curvature_position(binary_warped, ploty, left_fitx, right_fitx, left_fit, right_fit):
    '''
    Calculates the curvature of polynomial functions and vehicle position in meters.
    '''
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    ym_per_pix = 30/720       # meters per pixel in y-dimension
    xm_per_pix = 3.7/700      # meters per pixel in x-dimension
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    #print("left ", left_fit_cr, "right ", right_fit_cr)
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval* ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval* ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    y_max = binary_warped.shape[0]
    
    left_pos = left_fit[0]*y_max**2 + left_fit[1]*y_max + left_fit[2]
    right_pos = right_fit[0]*y_max**2 + right_fit[1]*y_max + right_fit[2]
    
    center_of_lane = (left_pos + right_pos)//2
    
    vehicle_pos = ((binary_warped.shape[1]//2) - center_of_lane) * xm_per_pix
    
    return left_curverad, right_curverad, vehicle_pos

### Step 7: Visual Display of lane curvature and vehicle offset on video

In [12]:
### Step 7 Adding text for lane info and displaying on image frames ###

def display_info(img, binary_warped, ploty, left_fitx, right_fitx, Minv, left_curve, right_curve, vehicle_position):
    # blank display to draw lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    points_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    points_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    points = np.hstack((points_left, points_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([points]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    
    # Combine the result with the original image
    out_img = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    cv2.putText(out_img,'Curve Radius [m]: '+str((left_curve+right_curve)/2)[:7],(40,70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.6, (255,255,255),2,cv2.LINE_AA)
    cv2.putText(out_img,'Center Offset [m]: '+str(vehicle_position)[:7],(40,150), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.6,(255,255,255),2,cv2.LINE_AA)
    
    return out_img

In [16]:
def pipeline_video(img):
    global left_fit_histogram
    global right_fit_histogram
    global previous_left_fit
    global previous_right_fit
    
    left_fit_histogram = np.array([])
    right_fit_histogram = np.array([])

    binary_threshold = pipeline(img)
    binary_warped, Minv = warp(binary_threshold)
    
    if (len(left_fit_histogram) == 0 or len(right_fit_histogram) == 0):
        leftx, lefty, rightx, righty, out_img = find_lane_pixels_histogram(binary_warped)
        left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
        # Store values in history
        left_fit_histogram = np.array(left_fit)
        new_left_fit = np.array(left_fit)
        left_fit_histogram = np.vstack([left_fit, new_left_fit])
        right_fit_histogram = np.array(right_fit)
        new_right_fit = np.array(right_fit)
        right_fit_histogram = np.vstack([right_fit, new_right_fit])
    
    else:
        previous_left_fit = [np.mean(left_fit_histogram[:,0]), np.mean(left_fit_histogram[:,1]), np.mean(left_fit_histogram[:,2])]
        previous_right_fit = [np.mean(right_fit_histogram[:,0]), np.mean(right_fit_histogram[:,1]), np.mean(right_fit_histogram[:,2])]
        leftx, lefty, rightx, righty = finding_lane_pixels_previous_info(binary_warped)
    
        if (len(lefty) == 0 or len(righty) == 0):
            leftx, lefty, rightx, righty = find_lane_pixels_histogram(binary_warped)
            left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
            # Store values in history
            new_left_fit = np.array(left_fit)
            left_fit_histogram = np.vstack([left_fit_histogram, new_left_fit])
            new_right_fit = np.array(right_fit)
            right_fit_histogram = np.vstack([right_fit_histogram, new_right_fit])
            # Remove old values in history
            if (len(left_fit_histogram) > 10):
                left_fit_histogram = np.delete(left_fit_histogram, 0, 0)
                right_fit_histogram = np.delete(right_fit_histogram, 0, 0)


    # visual_result = visualization(binary_warped, left_fitx, right_fitx, ploty)
    # Measuring Curvature and vehicle position:
    left_curve, right_curve, vehicle_pos = measure_curvature_position(binary_warped, ploty, left_fitx, right_fitx, left_fit, right_fit)
    #warp_result_NTU, Minv = warp(combined_binary)
    output_img = display_info(img, binary_warped, ploty, left_fitx, right_fitx, Minv, left_curve, right_curve, vehicle_pos)
    return output_img

video_output = 'challenge_video_output_3.mp4'
clip = VideoFileClip("challenge_video.mp4")
output = clip.fl_image(pipeline_video)
%time output.write_videofile(video_output, audio=False)


t:   0%|          | 2/485 [00:00<00:29, 16.26it/s, now=None]

Moviepy - Building video challenge_video_output_3.mp4.
Moviepy - Writing video challenge_video_output_3.mp4



Moviepy - Done !
Moviepy - video ready challenge_video_output_3.mp4
CPU times: user 7min 23s, sys: 28.4 s, total: 7min 52s
Wall time: 1min 34s
